In [2]:
import pandas as pd
from google.colab import drive
from pandas import Series, DataFrame
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns
from pandas import DataFrame
from pandas import concat
import os
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [4]:
raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')

In [5]:
raw_data

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0


In [6]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset = dataset[what_to_use]
  return dataset

In [13]:
dataset = drop_clms(raw_data)
dataset

,Time,DHI,DNI,RH,T,TARGET
0,0.0,0,0,69.08,-12,0.0
1,0.5,0,0,69.06,-12,0.0
2,1.0,0,0,71.78,-12,0.0
3,1.5,0,0,71.75,-12,0.0
4,2.0,0,0,75.20,-12,0.0
...,...,...,...,...,...,...
52555,21.5,0,0,70.70,-4,0.0
52556,22.0,0,0,66.79,-4,0.0
52557,22.5,0,0,66.78,-4,0.0
52558,23.0,0,0,67.72,-4,0.0


In [21]:
# 변수 2개, 앞의 변수는 raw_data 뒤에 변수는 timestep (96)

dataset2 = series_to_supervised(dataset, 96)
dataset2

,var1(t-96),var2(t-96),var3(t-96),var4(t-96),var5(t-96),var6(t-96),var1(t-95),var2(t-95),var3(t-95),var4(t-95),var5(t-95),var6(t-95),var1(t-94),var2(t-94),var3(t-94),var4(t-94),var5(t-94),var6(t-94),var1(t-93),var2(t-93),var3(t-93),var4(t-93),var5(t-93),var6(t-93),var1(t-92),var2(t-92),var3(t-92),var4(t-92),var5(t-92),var6(t-92),var1(t-91),var2(t-91),var3(t-91),var4(t-91),var5(t-91),var6(t-91),var1(t-90),var2(t-90),var3(t-90),var4(t-90),...,var5(t-7),var6(t-7),var1(t-6),var2(t-6),var3(t-6),var4(t-6),var5(t-6),var6(t-6),var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),var6(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var6(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),TARGET1(t+48),TARGET1(t+96)
96,0.0,0.0,0.0,69.08,-12.0,0.0,0.5,0.0,0.0,69.06,-12.0,0.0,1.0,0.0,0.0,71.78,-12.0,0.0,1.5,0.0,0.0,71.75,-12.0,0.0,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,...,-8.0,0.0,21.0,0.0,0.0,74.50,-8.0,0.0,21.5,0.0,0.0,74.50,-8.0,0.0,22.0,0.0,0.0,72.83,-8.0,0.0,22.5,0.0,0.0,72.82,-8.0,0.0,23.0,0.0,0.0,72.51,-8.0,0.0,23.5,0.0,0.0,72.49,-8.0,0.0,0.0,0.0
97,0.5,0.0,0.0,69.06,-12.0,0.0,1.0,0.0,0.0,71.78,-12.0,0.0,1.5,0.0,0.0,71.75,-12.0,0.0,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,-11.0,0.0,3.5,0.0,0.0,72.55,...,-8.0,0.0,21.5,0.0,0.0,74.50,-8.0,0.0,22.0,0.0,0.0,72.83,-8.0,0.0,22.5,0.0,0.0,72.82,-8.0,0.0,23.0,0.0,0.0,72.51,-8.0,0.0,23.5,0.0,0.0,72.49,-8.0,0.0,0.0,0.0,0.0,73.80,-8.0,0.0,0.0,0.0
98,1.0,0.0,0.0,71.78,-12.0,0.0,1.5,0.0,0.0,71.75,-12.0,0.0,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,-11.0,0.0,3.5,0.0,0.0,72.55,-11.0,0.0,4.0,0.0,0.0,74.62,...,-8.0,0.0,22.0,0.0,0.0,72.83,-8.0,0.0,22.5,0.0,0.0,72.82,-8.0,0.0,23.0,0.0,0.0,72.51,-8.0,0.0,23.5,0.0,0.0,72.49,-8.0,0.0,0.0,0.0,0.0,73.80,-8.0,0.0,0.5,0.0,0.0,68.20,-8.0,0.0,0.0,0.0
99,1.5,0.0,0.0,71.75,-12.0,0.0,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,-11.0,0.0,3.5,0.0,0.0,72.55,-11.0,0.0,4.0,0.0,0.0,74.62,-11.0,0.0,4.5,0.0,0.0,74.61,...,-8.0,0.0,22.5,0.0,0.0,72.82,-8.0,0.0,23.0,0.0,0.0,72.51,-8.0,0.0,23.5,0.0,0.0,72.49,-8.0,0.0,0.0,0.0,0.0,73.80,-8.0,0.0,0.5,0.0,0.0,68.20,-8.0,0.0,1.0,0.0,0.0,69.06,-8.0,0.0,0.0,0.0
100,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,-11.0,0.0,3.5,0.0,0.0,72.55,-11.0,0.0,4.0,0.0,0.0,74.62,-11.0,0.0,4.5,0.0,0.0,74.61,-11.0,0.0,5.0,0.0,0.0,73.74,...,-8.0,0.0,23.0,0.0,0.0,72.51,-8.0,0.0,23.5,0.0,0.0,72.49,-8.0,0.0,0.0,0.0,0.0,73.80,-8.0,0.0,0.5,0.0,0.0,68.20,-8.0,0.0,1.0,0.0,0.0,69.06,-8.0,0.0,1.5,0.0,0.0,69.04,-8.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52459,21.5,0.0,0.0,57.01,-1.0,0.0,22.0,0.0,0.0,56.82,-1.0,0.0,22.5,0.0,0.0,56.82,-1.0,0.0,23.0,0.0,0.0,56.90,-1.0,0.0,23.5,0.0,0.0,52.84,-1.0,0.0,0.0,0.0,0.0,52.59,-1.0,0.0,0.5,0.0,0.0,52.60,...,1.0,0.0,18.5,0.0,0.0,50.32,0.0,0.0,19.0,0.0,0.0,51.78,0.0,0.0,19.5,0.0,0.0,51.77,0.0,0.0,20.0,0.0,0.0,53.88,0.0,0.0,20.5,0.0,0.0,53.87,0.0,0.0,21.0,0.0,0.0,55.98,-1.0,0.0,0.0,0.0
52460,22.0,0.0,0.0,56.82,-1.0,0.0,22.5,0.0,0.0,56.82,-1.0,0.0,23.0,0.0,0.0,56.90,-1.0,0.0,23.5,0.0,0.0,52.84,-1.0,0.0,0.0,0.0,0.0,52.59,-1.0,0.0,0.5,0.0,0.0,52.60,0.0,0.0,1.0,0.0,0.0,52.19,...,0.0,0.0,19.0,0.0,0.0,51.78,0.0,0.0,19.5,0.0,0.0,51.77,0.0,0.0,20.0,0.0,0.0,53.88,0.0,0.0,20.5,0.0,0.0,53.87,0.0,0.0,21.0,0.0,0.0,55.98,-1.0,0.0,21.5,0.0,0.0,55.97,-1.0,0.0,0.0,0.0
52461,22.5,0.0,0.0,56.82,-1.0,0.0,23.0,0.0,0.0,56.90,-1.0,0.0,23.5,0.0,0.0,52.84,-1.0,0.0,0.0,0.0,0.0,52.59,-1.0,0.0,0.5,0.0,0.0,52.60,0.0,0.0,1.0,0.0,0.0,52.19,0.0,0.0,1.5,0.0,0.0,52.19,...,

In [18]:
dataset3 = seperator(dataset2)
dataset3

(array([[[ 1.10000000e+01,  1.04000000e+02,  9.41000000e+02,
           2.32900000e+01,  2.80000000e+01,  8.35927281e+01],
         [ 1.15000000e+01,  1.05000000e+02,  9.46000000e+02,
           2.32800000e+01,  2.80000000e+01,  8.59382031e+01],
         [ 1.20000000e+01,  1.06000000e+02,  9.47000000e+02,
           2.09600000e+01,  2.90000000e+01,  8.66878021e+01],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  4.31629072e+00,
           1.14474278e+01,  2.00794729e+01,  2.86173113e+01],
         [ 3.70609490e+01,  5.01959234e+01,  4.73806779e+01,
           6.10788540e+01,  6.07967163e+01,  2.79589838e+01]],
 
        [[ 2.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           7.49300000e+01,  7.00000000e+00,  0.00000000e+00],
         [ 2.50000000e+00,  0.00000000e+00,  0.00000000e+00,
           8.02900000e+01,  7.00000000e+00,  0.00000000e+00],

In [20]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 함수
def series_to_supervised(data, n_in=1, target = 'TARGET', dropnan=True):
    df = DataFrame(data)
    df2 = DataFrame(data[target])
    cols, names = list(), list()
    future = [48,96]
    n_vars = 1 if type(df) is list else df.shape[1]
    n_vars2 = 1 if type(df2) is list else df2.shape[1]
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(df2.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [8]:
def seperator(dataset):
  
  # series_to_supervised 함수를 지나며 전처리를 하면 하나의 row에 이전 7일간의 데이터가 일렬로 들어가고(train), 여기에 target으로 미래 96일 발전량이 따라 붙는다.
  # 이 둘이 한 줄에 있으므로 적절한 지점에서 잘라서 train_X와 train_y로 사용한다.
  X = dataset.iloc[:, :n_obs]
  y = dataset.iloc[:, -future_window:]
  
  # validation을 해야하니까, train으로 준 데이터를 train/test로 자른다.
  # 사용가능한 연도가 3개년이고, 대충 train에 2년치, test에 1년치를 주었다 (7:3)
  # 42 = ultimate answer to life the universe and everything
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

  # 학습과 검증 데이터를 원하는 모양([1,4,2]) 이런 식으로 바꾸는 함수
  #### 처음 모양 (36490, 2016)
  # 2년치가 데이터(730일 * 하루에 48틱 = 36490개)로 있고, 이전 일주일 데이터 7*48 = 336인데 사용한 변수가 6개라면 336*6이다.
  # 일단 2016개의 데이터를 틱별로(30분 단위로) 잘라준다
  # 한 틱에 6개의 변수가 들어간다면 이를 n_features로 반영
  
  #### 수정할 모양 (36490, 336, 6)
  # 6개 특징이 30분 단위로 336개(일주일) 준비되어 있다.
  # 이런 데이터가 36490개 있어서 학습에 사용할 수 있다.

  # y 데이터는 1개 변수(발전량)만 나오고, 미래 2일 (2*48 = 96틱)이므로
  # 형태 ('총 데이터의 개수', '미래/과거의 길이', '사용한 변수의 개수')
  # == (36490, 96, 1) 으로 reshape 한다.

  train_X = X_train.values
  train_X = train_X.reshape(train_X.shape[0],-1,n_features)
  train_y = y_train.values
  train_y = train_y.reshape(train_y.shape[0],future_window)

  test_X = X_test.values
  test_X = test_X.reshape(test_X.shape[0],-1,n_features)
  test_y = y_test.values
  test_y = test_y.reshape(test_y.shape[0],future_window)

  return train_X, train_y, test_X, test_y

In [9]:
def data_maker(train_X, train_y, test_X, test_y):
  train_X = torch.tensor(np.array(train_X), dtype=torch.float)
  train_X = train_X.transpose(1,0)
  train_y = torch.tensor(np.array(train_y), dtype=torch.float)

  test_X = torch.tensor(np.array(test_X), dtype=torch.float)
  test_X = test_X.transpose(1,0)
  test_y = torch.tensor(np.array(test_y), dtype=torch.float)
  return train_X, train_y, test_X, test_y

In [16]:
############ 파라미터 #####################
# 하루의 틱
ticks = 48
# 예측에 사용할 일수
days = 7
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']
# 사용할 변수
what_to_use = ['Time', 'DHI','DNI','RH','T','TARGET']

n_features = len(what_to_use)
n_obs = n_days * n_features

# 한 번에 뭉테기로 투입할 자료의 양
batches = 100
# 몇 번이나 반복하여 학습할 것인다.
epoch = 200

In [11]:
def data_loader(dataset):

  ############ 파라미터 #####################
  # 하루의 틱
  ticks = 48
  # 예측에 사용할 일수
  days = 7
  n_days = ticks*days

  # 미래 예측할 일수
  future_days = 2
  future_window = ticks * future_days

  ### 모든변수
  # ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']
  # 사용할 변수
  what_to_use = ['Time', 'DHI','DNI','RH','T','TARGET']

  n_features = len(what_to_use)
  n_obs = n_days * n_features


  dataset = drop_clms(dataset)
  dataset = series_to_supervised(dataset, n_days, future_window, target='TARGET')
  train_X, train_y, test_X, test_y = seperator(dataset)
  train_X, train_y, test_X, test_y = data_maker(train_X, train_y, test_X, test_y)
  return train_X, train_y, test_X, test_y